In [1]:
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import datetime
import cv2 as cv
import numpy as np
import os

In [2]:
CHECKPOINT_PATH="./trained_model/cp.ckpt"
EPOCHS_AMOUNT=5
CLASSES_AMOUNT=10
BATCH_SIZE=200

IMG_DIMENSIONS=28
NUMBER_THREAD=100

In [3]:
X = []
y = []
for i in range(10):
    for d in os.listdir("imgs/digits/{}".format(i)):
        t_img = cv.imread("imgs/digits/{}".format(i)+"/"+d)
        t_img = cv.cvtColor(t_img,cv.COLOR_BGR2GRAY)
        X.append(t_img)
        y.append(i)
        
X = np.array(X)
y = np.array(y)

In [4]:
input_shape = (IMG_DIMENSIONS, IMG_DIMENSIONS, 1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(CLASSES_AMOUNT, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [5]:
(image_train, labels_train) , (image_test, labels_test) = mnist.load_data()

In [ ]:
image_train

In [6]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state= 21)

In [7]:
image_train = image_train.reshape(image_train.shape[0],
                                    IMG_DIMENSIONS,
                                    IMG_DIMENSIONS,
                                    1)
image_train = image_train.astype(np.float32) / 255

image_test = image_test.reshape(image_test.shape[0],
                                    IMG_DIMENSIONS,
                                    IMG_DIMENSIONS,
                                    1)
image_test = image_test.astype(np.float32) / 255

labels_train = utils.to_categorical(labels_train, 
                                        num_classes=CLASSES_AMOUNT)
labels_test = utils.to_categorical(labels_test, 
                                        num_classes=CLASSES_AMOUNT)


In [8]:
checkpoint_dir = os.path.dirname(CHECKPOINT_PATH)

cp_callback = ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                save_weights_only=True,
                                verbose=1)
model.fit(image_train,
            labels_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS_AMOUNT,
            verbose=1,
            validation_data=(image_test, labels_test),
            callbacks=[cp_callback])

Epoch 1/5
300/300 [==============================] - ETA: 0s - loss: 0.2758 - accuracy: 0.9169
Epoch 00001: saving model to ./trained_model\cp.ckpt
300/300 [==============================] - 64s 212ms/step - loss: 0.2758 - accuracy: 0.9169 - val_loss: 0.0580 - val_accuracy: 0.9815
Epoch 2/5
300/300 [==============================] - ETA: 0s - loss: 0.0958 - accuracy: 0.9717
Epoch 00002: saving model to ./trained_model\cp.ckpt
300/300 [==============================] - 66s 221ms/step - loss: 0.0958 - accuracy: 0.9717 - val_loss: 0.0427 - val_accuracy: 0.9861
Epoch 3/5
300/300 [==============================] - ETA: 0s - loss: 0.0730 - accuracy: 0.9778
Epoch 00003: saving model to ./trained_model\cp.ckpt
300/300 [==============================] - 61s 204ms/step - loss: 0.0730 - accuracy: 0.9778 - val_loss: 0.0372 - val_accuracy: 0.9883
Epoch 4/5
300/300 [==============================] - ETA: 0s - loss: 0.0583 - accuracy: 0.9817
Epoch 00004: saving model to ./trained_model\cp.ckpt
300/30

In [9]:
model.evaluate(image_test,  labels_test, verbose=2)[1]

313/313 - 3s - loss: 0.0323 - accuracy: 0.9890 - 3s/epoch - 9ms/step


0.9890000224113464

In [10]:
image = cv.imread('imgs/')
plt.imshow(image)

AttributeError: 'NoneType' object has no attribute 'reshape'

In [13]:
model.save("trained_model")

INFO:tensorflow:Assets written to: trained_model\assets


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit